In [20]:
# Imagine we have a coin and we want to test whether it’s fair. 
# We’ll make the assumption that the coin has some probability p of landing heads, 
# and so our null hypothesis is that the coin is fair


# In particular, our test will involve flipping the coin some number n times and counting the number of heads X.
# Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable, 
# which we can approximate using the normal distribution:

def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [21]:
# Whenever a random variable follows a normal distribution, we can use normal_cdf 
# to figure out the probability that its realized value lies within (or outside) a particular interval:

def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# the normal cdf _is_ the probability the variable is below a threshold”

normal_probability_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [22]:
# We can also do the reverse—find either the nontail region or the (symmetric) interval around the mean
# that accounts for a certain level of likelihood.

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds “that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [23]:
# In particular, let’s say that we choose to flip the coin n=100 times. If our hypothesis of fairness is true, 
# X should be distributed approximately normally with mean 50 and standard deviation 15.8

import math 

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [24]:
# We need to make a decision about significance—how willing we are to make a type 1 error (“false positive”), 
# in which we reject h0 even though it’s true. For reasons lost to the annals of history, this willingness is often 
# set at 5% or 1%. Let’s choose 5%.

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z


normal_two_sided_bounds(0.95, mu_0, sigma_0)   # (469, 531)

(469.01026640487555, 530.9897335951244)

In [25]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability      # 0.887”

In [26]:
# Imagine instead that our null hypothesis was that the coin is not biased toward heads. 
# In that case we want a one-sided test that rejects the null hypothesis when X is much larger than 50 
# but not when X is smaller than 50. So a 5%-significance test involves using normal_probability_below
# to find the cutoff below which 95% of the probability lies:

hi = normal_upper_bound(0.95, mu_0, sigma_0)
# is 526 (< 531, since we need more probability in the upper tail)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability      # 0.936”

In [27]:
# For our two-sided test of whether the coin is fair, we compute

def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)
    
# If we were to see 530 heads, we would compute:

two_sided_p_value(529.5, mu_0, sigma_0)   # 0.062” 

0.06207721579598857

In [28]:
import random

extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # count # of heads
                    for _ in range(1000))                # in 1000 flips
    if num_heads >= 530 or num_heads <= 470:             # and count how often
        extreme_value_count += 1                         # the # is 'extreme'

print (extreme_value_count / 100000)   # 0.062

0


In [29]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [30]:
upper_p_value(524.5, mu_0, sigma_0) # 0.061”
upper_p_value(526.5, mu_0, sigma_0) # 0.047”

0.04686839508859242

In [31]:
# Confidence intervals 

# we can estimate the probability of the unfair coin by looking at the average value of the Bernoulli variables 
# corresponding to each flip—1 if heads, 0 if tails. If we observe 525 heads out of 1,000 flips, then we estimate
# p equals 0.525. How confident can we be about this estimate?. If we knew the exact value of p, 
# the central limit theorem tells us that the average of those Bernoulli variables should be approximately normal,
# with mean p and standard deviation:

import math

math.sqrt(p * (1 - p) / 1000)

NameError: name 'p' is not defined

In [32]:
# Here we don’t know p, so instead we use our estimate:”

p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)   # 0.0158

In [33]:
# This is not entirely justified, but people seem to do it anyway. 
# Using the normal approximation, we conclude that we are “95% confident” that the following
# interval contains the true parameter p:

normal_two_sided_bounds(0.95, mu, sigma)        # [0.4940, 0.5560]

(0.0, 0.0)

In [34]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000) # 0.0158
normal_two_sided_bounds(0.95, mu, sigma) # [0.5091, 0.5709]

(0.0, 0.0)

In [35]:
# “P-hacking

# A procedure that erroneously rejects the null hypothesis only 5% of the time will—by definition—5% of the time 
# erroneously reject the null hypothesis:

def run_experiment():
    """flip a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

print num_rejections   # 46

46


In [36]:
# Example: Running an A/B Test

# If 990 out of 1,000 A-viewers click their ad while only 10 out of 1,000 B-viewers click their ad, 
# you can be pretty confident that A is the better ad. But what if the differences are not so stark?
# Here’s where you’d use statistical inference.

def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [37]:
# This means we can test the null hypothesis that pA and pB are the same (that is, that pA-pB  is zero), using the statistic:

def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [39]:
# For example, if A gets 200 clicks out of 1,000 views and B gets 180 clicks out of 1,000 views, the statistic equals:

# z = a_b_test_statistic(1000, 200, 1000, 180)    # -1.14

In [43]:
# Bayesian Inference

# For example, when the unknown parameter is a probability (as in our coin-flipping example),
# we often use a prior from the Beta distribution, which puts all its probability between 0 and 1:

def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:          # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)
    alpha / (alpha + beta)